# Jupyter Snake

---

In this project our goal is to beat the game of snake, applying multiple RL techniques in order to teach an agent how to play the game.\
The project is divided in 3 parts:
1. Developement of the Environment
2. Implementation of the Algorithms
3. Learning and evaluating phase of the Algorithms

This whole project is developed as final project for the "Reinforcement Learning" course (2024-2025).

Authors : *Bredariol Francesco, Savorgnan Enrico, Tic Ruben*

In [ ]:
from algorithms import *
from eligibility_traces import *
from epsilon_scheduler import * 
from snake_environment import *
from states_bracket import *
from utils import *

## PART 1
---
*Environment*

### **The Game**

For who who doesn't know Snake is a not just a game but a genre of action video games.\
It was born in 1976 competitive arcade video game Blockade, where the goal was to survive longer than others players while collecting the most possible food.\
In this game you control the head of a snake on grid world and you aim to eat some food in order to become bigger. The big difficulty here is that if you hit your tail (this is the only common rule for all snake variant) you die.\
There are multiple version of the game and some of them are really weird (where teleportation can occour or some food can actually make you die).\
We took in account the most basic version, where:

1. The world is a discrete grid world of size $n\times m$
2. There is always only one food (apple) on the grid world, and when you it changes position
3. There are no periodic boundary conditions, meaning that if you hit a wall you die

The rest is just as described as in the introduction to the game.\
Little side note is that this version is inspired by the "Snake Byte" published by Sirius Software in 1982.

### **The Implementation**

Thanks to Gymnasium and PyGame the implementation of this simple version of the game is pretty straightforward.\
Developed completely in Python in the file "snake_environment.py", this implementation follows the Gym protocol, defining:

1. Step
2. Reset
3. Render, which use PyGame
4. Close

All others functions are private and only used inside the class, for the exception of "get_possible_action" which is useful for our purpose.

One important thing is that we actually defined a maximum number of step inside the environment to prevent infinte loop driven by bad policies while training. This is a parameter for the __init__ with default value 1000.

In [ ]:
import random
env = SnakeEnv(render_mode="human")

done, keep = False, True

state, _ = env.reset()
action = 0

while not done and keep:
    action = random.choice(env.get_possible_actions(action))
    state, reward, done, trunc, inf = env.step(action)
    keep = env.render()

env.close()

### **The Dimensionality Problem**

Once the environment is defined, one can think about how big the space of all possible configuration is.\
Well, doing some (pretty bad but although valid) approximation, considering a state as the matrix representation of the grid with 0 (empty cell), 1 (apple), 2(head) and 3 (tail), the dimension of all possible configuration ends up being something like this:
$$
    |S| = (n\times m)(n\times m)2^{(n\times m)}
$$
This should describe all the possible positions of the apple, all the possible position of the head and all possible configuration on the grid of the tails (now this is the big approximation, since the tail configuration is not independent from the head position). Anyway, even if this is an approximation one can simply add the "blocks" on the grid world (static cells that kill, if touched, the snake) and the dimension should exactly being that big.

Now this is not a simple thing to deal with while learning. Solution? Binning (or bracketing, how we actually call it). Now on this soon.

## PART 2
---
*Algorithms*

## **BINNING (or Bracketing)**

As shown before the Snake game has a huge state dimension.\
Since a look up table of that dimension has no logic to exist (I don't even think our computers can store something like that) and since it is pretty impossible just to see one example for each pair state-action in an entire life time, something had to come in mind.\
Thanks to a lesson we learnt about binning (that we called bracketing for the entire project, shame on Francesco) and decided to try it out.\
Binning essentially is just a supervised technique (it is the human that codifies how it works) that agglomerates similar states together, in order to reduce the dimensionality of the problem.\
A very stupid example could be the following: each state just randomly be labelled as 0 or 1. And the agents now will not see the entire state representation, but only the label you gave it. Now this example is stupid because, using this random strategy, you end up with no knowledge at all. BUT, look at what happened at your state dimension: it fell down from whatever it was to 2! Pretty neat, uh?

We discussed together end decided to try a lot of different binning techniques, and we end up discovering the big tradeoff in this field: **information against dimension**.\
Minimal state binning are easy to implement, but if they are too small it is not ensured that they will bring enough information to actually learn to the agent. On the other hand, if you give too many information to the agent, you will end up again with a too big state dimension to deal with.

Another important aspect of binning is that you actually "completely" lose the transition function below your system. It is true that you can develop a new transition function on a new space, the "binning space", but it is not easy and it is not ensured to be relevant.

Now we will quickly see all the binnings we implemented. We defined in the states_bracket.py a super class (StateBracket) which implement the protocol for all the binning techniques.

### **FOOD INFORMATION ONLY**

Our first approach was to give only information about the position of the apple wrt the head of the snake. Two variants of this idea came up:

1. Food Relative Position 
2. Food Direction

Both of the techniques are pretty straightforward to implement, and their computation is very fast as well. However, the snake loses information about its tail, the walls and the boundaries of the cell.


##### **Food Relative Position**

Once you get the apple position $(a_y, a_x)$ and the head position $(h_y, h_x)$ you just return as state the tuple $(a_y - h_y, a_x - h_x)$. This approach reduces the states' dimension up to $2m \times 2n$

##### **Food Direction**

Once you get the apple position $(a_y, a_x)$ and the head position $(h_y, h_x)$ you just return as state the tuple $(a_y < h_y, a_y > h_x, a_x < h_x, a_x > h_x)$.\
Now, while the first one is straightforward, this a little more subtle. This tells you whether your head is above, below, to the left, or to the right of the food. \
This is a very minimal information: the states are condensed into just $8$ bins.



In [ ]:
state = np.array([
    [0, 0, 0], 
    [0, 0, 1], 
    [0, 0, 2]
    ])
frp = FoodRelativePositionBracket()
fd = FoodDirectionBracket()
print(frp.bracket(state))
print(fd.bracket(state))


These binning techniques combines the things we have just seen plus information relative to the neighborhood of the head.\
Neighborhood could be both of Von Neumann ('V') type or Moore ('M') type.
A Von Neumann neighborhood is defined as the four cells that are vertically or horizontally adjacent to the head of the snake.
On the other hand, a Moore neighborhood considers vertically, horizontally and diagonally adjacent cells.

The radius is a parameter for this binning. Notice that the greater the radius, the greater the total state dimension. The information within the neighborhood are expressed in the form of 0, 1, 2 and 3. 0 if a cell is free, 1 if a cell is occupied by the food, 2 if is occupied by the head and 3 if is occupied by the tail.

In [ ]:
state = np.array([
    [0, 0, 0], 
    [0, 0, 1], 
    [0, 0, 2]
    ])
npfrp = NeighPlusFoodRelativePositionBracket(neigh='V', radius=1)
npfd = NeighPlusFoodDirectionBracket(radius=1)
print(npfrp.bracket(state))
print(npfd.bracket(state))

### **ONLY NEIGHBORHOOD INFORMATION**

An interesting experiment was the definition of the binning containing only information relative to the neighborhood of the head (adding the 2 value for the apple in this case).\
The idea was that, using this type of binning, the agent could have learnt to search being really careful, but probably with no knowledge on its own position it is impossible for it to learn a valid strategy.

In [ ]:
state = np.array([
    [0, 0, 0], 
    [0, 0, 1], 
    [0, 0, 2]
    ])
n = NeighborhoodBracket()
print(n.bracket(state))

### **FOOD AND NEIGHBORHOOD AND TAIL INFORMATION**

The last type of binning explored combines all the informations described until now plus a little information about the tail.\
The relative information about the tail is its length, and should help the agent learn to be a little more careful when the tail gets longer.

In [ ]:
state = np.array([
    [0, 0, 0], 
    [0, 0, 1], 
    [0, 0, 2]
    ])
tpnpfrp = NeighPlusFoodRelativePositionPlusTailBracket()
tpnpfd = NeighPlusFoodDirectionPlusTailBracket()
print(tpnpfrp.bracket(state))
print(tpnpfd.bracket(state))

## ALGORITHMS

### **Why not MDP**


How we have seen in the latest section, the only way to obtain a dealable state dimension is using the binning. \
The problem is that, using binning, we lose the transition function below our MDP. Indeed, being in the same state, taking the same action, ending into the same state, could possibly lead to different rewards.
To give an example, consider the two situations below, where we marked with a $O$ the head of the snake, with $0$ its body and with $X$ the food (consider the snake without tail):


\begin{array}{|c|c|c|}
\hline
 & 0 & 0 \\
\hline
 & O & 0 \\
\hline
X &  &  \\
\hline
\end{array}

\begin{array}{|c|c|c|}
\hline
 &  &  \\
\hline
 & O & 0 \\
\hline
X &  & 0 \\
\hline
\end{array}


Such situations are binned into the same condensed-state $(-1, -1)$. However, the action "UP" will lead in the first example to a negative reward because of the hit of the snake's tail, while  in the second case, to a zero reward.

This erases the capability to retrieve good results solving the problem as an MDP via policy iteration.

TO ARGUMENT A BIT MORE


### **Our Choices**


In the end we decided to develop 5 different **Model-free algorithms** (with also some variants of them) in order to take familiarity with the whole RL framework. These are the algorithms we implemented:

1. Montecarlo 
2. SARSA
3. QLearning
4. DDQL
5. Policy Gradient

We firstly implemented a super class that defines a protocol for all the algorithms and provides useful function such as "get_action_during_learning" and so on.\
Useful method of the class are the save and upload methods, which can let to store the results obtained as a pickle dictionary in order to retrieve it later.\
In addiction the utils.py contains a lot of useful function used to deal with the default_dict, a structure we used to store the QValues look up table (for the algorithms that require it). Since we used a lot of binning we used default dict to possibly deal with no-fixed state dimension.

Before diving in our actual implementation, let's briefly remind some key concepts that will be used.

*Credit for the key concepts to [this tutorial](https://medium.com/@hsinhungw/intro-to-reinforcement-learning-monte-carlo-to-policy-gradient-1c7ede4eed6e)*

### **Policy-based vs Value-based**

Most of our algorithms are value-based but policy gradient is in fact policy-based. Let's define the difference between these two classes.

1. **Policy-based methods**: The agent learns the policy directly.
2. **Value-based methods**: The agent learns a value function that gives the expected return of being in a given state or performing a given action in a given state. The policy can then be derived from the learned value function.

### **Off-policy / On-policy**

In RL, the agent generates experience by interacting with the environment under a certain policy and then learns the policy from those experiences. Given this in mind, algorithms can use different approaches to the policy. Two main classes exist, let's break them down.


1. **On-policy methods**: The agent attempts to learn the policy that is also used to generate the experience.
2. **Off-policy methods**: The agent learns a policy that is different from the one used to generate experiences.

### **Q-Value Learning**


Our goal is to find the optimal policy given only the experience of an environment. Again, the experience consists in the trajectories we perform as we explore the environment

$\tau_i = (S^i_0, A^i_0, R^i_1, S^i_1, A^i_1, R^i_1, \dots S^i_{T^i})$


To do this, it will be more convenient to consider, instead of the _state_-function $V_{\pi} (s) $, the _state/action_-function $Q_{\pi}(s,a)$. It is defined as:

$$
Q_{\pi}(s,a) = \mathbb{E}_\pi\bigg[ \sum_{t=0}^\infty \gamma^t \, R_{t+1} \, \, \Big| \, \, S_0 = s, A_0 = a \bigg] \ .
$$

**Definition 1** _"The expectation discounted cumulative sum of all future rewards when starting from state $s$, acting with action $a$ and then always following the policy $\pi$"_

or equivalently as:

$$
Q_\pi(s,a) = \mathbb{E}_{\pi}\bigg[ R_1 + \gamma V_\pi(S_1) \,\, \Big| \,\, S_0 = s, A_0 = a\bigg] \ .
$$

**Definition 2** _"The expectation value of the immediate reward plus the discounted $V$-value of the following state $S_1$, when starting from state $s$, acting with action $a$ and then always following the policy $\pi$."_

or again as:

$$
Q_\pi(s,a) = \mathbb{E}_{\pi}\bigg[ R_1 + \gamma  \, \sum_{a'} \, Q_\pi(S_1, a') \, \pi(a' | S_1) \,\, \Big| \,\, S_0 = s, A_0 = a\bigg] \ .
$$
**Definition 3** _"The expectation value of the immediate reward plus the discounted $Q$-value of the following state $S_1$ and all possible actions $a'$ weighted by the probability of taking that action ($\pi(a' | S_1)$), when starting from state $s$, acting with action $a$ and then always following the policy $\pi$."_

*Credit to the 4th tutorial for the definition*

### **Montecarlo**



Monte Carlo methods are ways of estimating value functions through experiences (sample sequence of states, actions, and rewards). Recall that the value of a state is the expected return (expected cumulative future discounted reward) starting from that state. One way to estimate the expected return is simply to average the returns observed after visits to that state. As more returns are observed, by the law of large numbers, the average should converge to the expected return. This idea underlies all Monte Carlo methods.

Our first algorithm developed was the **on-policy Montecarlo**. It works as follows:

1. On-policy MC uses an **ε-soft policy**. A soft policy refers to any policy that has a small, but finite, probability of selecting any possible action, ensuring exploration of alternative actions.
2. After each episode, the **observed returns** are used to learn the q-value function, and then the policy is improved based on the learned value function for all the states visited in the episode.

This means that we use the first definition. This definition has no bias but an high variance.

![onpolicymc](./images/on_policy_mc_correct.png)

In [ ]:
env = SnakeEnv(render_mode="non-human", max_step=100)
mc = Montecarlo(action_space=4, gamma=0.90, lr_v=0.01)
eps = ConstantEpsilonDecay(0.3)
bracketer = NeighPlusFoodDirectionBracket()
mc.learning(env = env, epsilon_schedule= eps, n_episodes=400, bracketer=bracketer)
env = SnakeEnv(render_mode="human", max_step=2000)
mc.play(env=env, bracketer=bracketer)

### **SARSA**

SARSA is a fascinating on-policy method which use state action and reward (State Action Reward State Action) in order to achieve its goals.

We implemented both a SARSA(0) version, where the Q-values are updated each step, and a SARSA(λ). \
The second option, despite being a bit slower, is a more general and flexible algorithm which leaves the possibility to see more than one action at the time, so to have a more accurate update of the Q-values. In this version of SARSA we implemented a "backward approach" thanks to eligibility traces.

The (common) pseudocode for SARSA(0) is here below:

![sarsa](./images/sarsa.png)

In [ ]:
env = SnakeEnv(render_mode="non-human", max_step=100)
sarsa = SARSA(action_space=4, gamma=0.90, lr_v=0.01)
eps = ConstantEpsilonDecay(0.3)
bracketer = NeighPlusFoodDirectionBracket()
sarsa.learning(env = env, epsilon_schedule= eps, n_episodes=500, bracketer=bracketer)
env = SnakeEnv(render_mode="human", max_step=2000)
sarsa.play(env=env, bracketer=bracketer)

In [ ]:
env = SnakeEnv(render_mode="non-human", max_step=100)
sarsal = SARSALambda(action_space=4, gamma=0.90, lr_v=0.01, lambda_value=0.1)
eps = ConstantEpsilonDecay(0.3)
bracketer = NeighPlusFoodDirectionBracket()
sarsal.learning(env = env, epsilon_schedule= eps, n_episodes=500, bracketer=bracketer)
env = SnakeEnv(render_mode="human", max_step=2000)
sarsal.play(env=env, bracketer=bracketer)

### **QLearning**

QLearning is the first off-policy algorithm we see. 

In [ ]:
env = SnakeEnv(render_mode="non-human", max_step=1000)
ql = QLearning(action_space=4, gamma=0.90, lr_v=0.01)
eps = LinearEpsilonDecay(1, 0.999, 0.2)
bracketer = NeighPlusFoodDirectionBracket()
ql.learning(env = env, epsilon_schedule= eps, n_episodes=20000, bracketer=bracketer)
env = SnakeEnv(render_mode="human", max_step=2000)
ql.play(env=env, bracketer=bracketer)

In [ ]:
ql.play(env=env, bracketer=bracketer)

### **Double Deep Q Learning (DDQL)**

This algorithm is a variant of the Q-Learning algorithm, which is an off-policy method.

It is basically a combination of Q-Learning and Deep Q-Networks (DQN), used to estimate the Q-values.
The main idea behind DDQL is to use two separate neural networks for the estimation, which helps to reduce the overestimation bias that can occur in Deep Q-learning and in Q-learning in general.
Indeed, in common Q-Learning, the use of the $\max$ operator may lead to inflated Q-values. This consequently leads to suboptimal policies, especially in complex environments.

DDQL addresses the problem by exploiting a **Online neural network** $\tilde{Q}$ for choosing the best action given a certain state, and using a **Target neural network** $\hat{Q}$ for computing the estimated Q-values given the couple (state, action).
The target network is updated less frequently than the online one, so the two are basically decoupled, and their predictions are independent. This helps to stabilize the learning process and reduce the overestimation bias.

Given the sampled batch $\mathcal{B}$ , the Mean Squared Error (MSE) loss used is calculated as:
$$
    \mathcal{L} = \bigl(\sum_{i \in \mathcal{B}} \tilde{Q}(s_i, a_i, \tilde{\theta}) - (R_i + \gamma*\hat{Q}(s_i, a^*, \hat{\theta}) \bigr)^2,
$$
where $a^*$ is:
$$
    a^* := \arg\max_a \tilde{Q}(s_i, a, \tilde{\theta})
$$

A key aspect of DDQL is the use of a **memory buffer** that stores all the tuples ```(state, action, reward, next state)``` occurred during the episodes.
The main idea behind the memory buffer is that it breaks correlations between near updates of the online neural network: without it, the updates would be highly correlated, and the neural net would probably overfit and lose information about its past. Moreover, it leads to improve exploration of the environment
At each iteration, if the memory buffer stores a right amount of history, a small batch of a given size is sampled and used for the learning step.

The main advantage of the use of this architecture is that it allows to manage bigger and more general binnings, closer to the un-binned state.
However, it needs up to 4x more time to train compared to policy gradient algorithms.

![ddql](./images/ddql.png)

In [ ]:
env = SnakeEnv(render_mode="non-human", max_step=100)
ddql = DeepDoubleQLearning(
    action_space=4,
    state_dim=bracketer.get_state_dim(),
    gamma=0.90,
    lr_v=0.001,
    batch_size=128,
    memory_size=10000,
    target_update_freq=200,
    device="cuda" if torch.cuda.is_available() else "cpu")
eps = ConstantEpsilonDecay(0.3)
bracketer = NeighPlusFoodDirectionBracket()
ddql.learning(env = env, epsilon_schedule= eps, n_episodes=2000, bracketer=bracketer)
env = SnakeEnv(render_mode="human", max_step=2000)
ddql.play(env=env, bracketer=bracketer)

#### Main results

Several DDQL architectures have been trained.

For all of them, the following hyperparameters are kept fixed:
- *ε*: we chose a linearly-decaying epsilon for the ε-greedy policy, starting from ε=1 to ε=0.05, with a slope of -0.999.
- *learning rate*: constant at 0.001
- *memory size*: the maximum size of the memory buffer, fixed to 10000.
- *batch size*: the number of samples from the memory buffer, fixed to 128.
- *frequency of update of the target neural network*: 200 step.

The following bins have been considered:
- Von Neumann Neighborhood + Food Direction
- Von Neumann Neighborhood + Food Relative Position
- Von Neumann Neighborhood + Food Direction + Tail, with radius 1
- Von Neumann Neighborhood + Food Direction + Tail, with radius 5
- Von Neumann Neighborhood + Food Relative Position + Tail, with radius 1
- Von Neumann Neighborhood + Food Relative Position + Tail, with radius 5
- Moore Neighborhood + Food Direction + Tail, with radius 5
- Moore Neighborhood + Food Relative Position + Tail, with radius 5

Each model have been trained using 5000 episodes. This because, as shown by the average reward plots during the training, almost all the models near to 3500~4000 episodes do not show any improvement in the mean return.
To avoid any doubt, a model has been trained for 20000, without any relevant improvement in terms of efficiency and results.
The different models required from 40 minutes up to 1 hour and a half to train for 5000 episodes using GPUs.

A key aspect noticed during training is that the use of different bins lead to very similar results. On average all the bins lead the snake to eat on average 6 pellets, that is not an excellent result. A reason for such performance is that at the beginning, if the snake spawns near the boundary of the grid, it easily tends to prefer a direct suicide more than moving towards food.

Deeper results of DDQL training will be discussed during the presentation.

## **Policy Gradient**

Up untill now, we focused on determining the state-action value function Q. The policy was only a subproduct of Q. \
We might wonder whether it is possible to approximate the optimal policy **directly**.
The answer is Policy Gradient.

### **Theory**

#### Policy Gradient Theorem

Let G be the average return. Let $ \pi{_\theta}$ be a parametrized policy. Then we have that: 
$$
\nabla_\theta G = \sum_{s a} \pi(a|s)\,\eta_\pi(s)\,Q(s, a)\; \nabla_\theta \log(\pi(a|s)) 
$$
The powerful aspect of this equation is this: the summatory is obtained expermentally. The algorithm needs only to consider $Q(s, a)\; \nabla_\theta \log(\pi(a|s))$ for every step. In fact we might rewrite the same equation, up to a constant factor, as follows: 
$$
\nabla_\theta G = E_\pi \left[ Q(S_t, A_t)\; \nabla_\theta \log(\pi(A_t|S_t))\qquad | \qquad S_0 \sim \rho_0 \; A \sim \pi \right]\qquad (*)
$$

From now on we will use the **Softmax parametrization** in a tabular contex. 


#### Actor Only

We can use the equation (*) directly. Since the policy must remain fixed as we calculate the gradient, we update at the end of each episode. Thus, we can use a Monte Carlo estimator for $Q$. Here we show a simple pseudocode:

collect  $\{S_t, A_t, R_t\}_{t= 0, 1, ..., T}$ 

$G = 0$ \
for $t = T, T-1, ... 0$ \
$\qquad G = R_t + \gamma G$ \
$\qquad \theta = \theta +  \alpha G \nabla_\theta \log(\pi(A_t | S_t))$

Here $\alpha$ is a fixed learning rate. In the case of softmax, we have that $\theta = \{ \theta_{sa}\} \text{ with } s \in S \; a \in A$. Then, the gradient is 
$$
\nabla_\theta \log(\pi(A_t | S_t))_{\theta_{s,a}} = 
\begin{cases}
1 - \pi(A_t | S_t)\qquad &s, a = S_t, A_t \\
-\pi(A_t | S_t) \qquad &s = S_t \; a \neq A_t \\
0 \qquad &\text{else}
\end {cases}
$$

There is an intuitive way to interpret this code: if, in a certain state $s$, an action $a$ yielded good results, we increase $\theta_{s, a}$ and decrease $\theta$ of the other actions in the same state.


#### Actor Critic

Actor Only is a very simple, but not efficient approach. The main problem is that we value an action through the return obtained afterwards, but *without a contex*. A same return can be very suboptimal, if the sistem is generous, or very good, if rewards are rare. A better and equivalent way to estimate $\nabla_\theta G$ is through the *advantage*, defined as:
$$
Ad(s, a) = Q(s, a) - V(s)
$$

If we use bootstrapping we can write 
$$
Ad(s, a) = R_t + \gamma V(S_{t+1}) - V(S_t) = \delta_t
$$

And we already have a way to estimate $V(s)$: Temporal Different Learning. We can combine the two, obtaining what is called Actor Critic. In fact we have a Critic, which estimates $V$, and an Actor, which estimates $\pi$ through gradient ascent. We present here a simple pseudocode:

collect $S_t, A_t, R_t, S_{t+1}$ 

$\delta = R_t + \gamma V(S_{t+1}) - V(S_t)$ \
$V(S_t) = V(S_t) + \alpha_V \delta $

$\theta = \theta +  \alpha_A \delta \nabla_\theta \log(\pi(A_t | S_t))$

For the learning rates, we ask that $\alpha_V > \alpha_A$. In fact the Actor is using the Critic for its estimation, thus the Critic must learn faster than the Actor.


#### General Advantage Estimation

With Actor Critic algorithm we have to update the policy at every step. This can be time consuming. Furthermore, it could be better to update $\pi$ only at episode ended, so that we can have a better idea on the long term performance of the policy. For that we use an algorithm called General Advantage Estimation (GAE).

So far we estimated $Ad$ with only the current reward. But, since we update at the end of the episode, we can actually consider $n$ future rewards, obtaining:

$$
Ad_t^n = R_t + \gamma R_{t+1} + ... + \gamma^n R_{t + n} + \gamma^{n+1} V(S_{t+n+1}) \;- \; V(S_t) = [R_t -  V(S_t)] \; +\; \gamma R_{t+1} + ... + \gamma^n R_{t + n} + \gamma^{n+1} V(S_{t+n+1}) \\
       = \delta_t + \gamma[ R_{t+1} - V(S_{t+1})] + ... + \gamma^n R_{t + n} + \gamma^{n+1} V(S_{t+n+1}) = ... = \delta_t + \delta_{t+1} + ... + \delta_{t+n}
$$


The idea behind GAE is to consider $Ad^n$ for all $n$, each discounted by $\lambda^n$. Specifically 
$$
Ad_t^{GAE} = (1-\lambda) \sum_{n = 0}^{\infty} \lambda^nAd_t^n = (1-\lambda) \left[ \delta_t(1 + \lambda + \lambda^2 + ...) + \gamma \delta_{t+1}(\lambda + \lambda^2 + ...) + ...\right] = \sum_{n = 0}^{\infty} (\lambda\gamma)^n\delta_{t+n} 
$$

The advantage can be seen also in another way. If we conider the average of $Ad$ with fixed state-action, we obtain the advantage. Instaed, if we consider the average of $Ad$ with a fixed state, we obtain a measure of how much our estimator is correct. Indeed we will use $Ad$ both for estimating $V$ and $\pi$. Here we present a pseudocode: 

collect  $\{S_t, A_t, R_t\}_{t= 0, 1, ..., T}$ 

$Ad = 0$ \
for $t = T, T-1, ... 0$ \
$\qquad \delta = R_t + \gamma V(S_{t+1}) - V(S_t) $\
$\qquad Ad = \delta_t + \lambda\gamma Ad$ 

$\qquad V(S_t) = V(S_t) + \alpha_V\, Ad$

$\qquad \theta = \theta +  \alpha_A\; Ad \;\nabla_\theta \log(\pi(A_t | S_t))$

With this algorithm we have an interesting variabile in hand: $\lambda$. If $\lambda$ is close to 0, we learn only from close rewards. Thus we will have less variance, but there will be a bias due to bootstrapping. If, on the contrary, $\lambda$ is close to 1, distance rewards have a great influence. Thus variance will increase, but there will be less bias. In fact we are moving toward a Monte Carlo apprach, notoriously unbiased.

A little note: the algorithm presented here is actually a variation of the standard GAE. In fact we are calculating the advantages, and at the same time updating the values. Usually the advantages are calculated first, and then the values are updated. This is clearly a convenient approach for batch training with Neural Networks. Here we are not using NNs, and then we get to choose one of the two approaches. The "GAE in place" algorithm has been chosen because it exhibits lower variance. Infact $\delta_t$ is calculated with a value function already updated.

### **Results**

For all the results shown in this chapter, we used as bin the 8 directions of food (sud, sud-est, est, ...) and a neighborhood of the snake's head. The total number of bins can be calculated in this way: 
$$
N = n_{\text{food directions}} \cdot 2^{\text{size neighborhood}} = 8 \cdot 2^{\text{size neighborhood}}\qquad (*)
$$

As a target return, we have built a simple reference policy. It sais: don't go against a block, if possible go towards food, else go randomly in a free direction. For all examples, unless stated otherwise, we set reward food = 20, penality step = 0.2, penality death = 10. With this data, the average performance of this policy is around 400. 

#### Using Von Neuemann neighborhood radius 1

For this part, we have learned policies using as bin the Von Neuemann radius 1 neighborhood plus the direction of food. This binning has been the most successfull one, since the number of different bins is small enough for a thorough learning. Specifically, using (*) the total number of bins is 128. 

Let us start with the most successfull learining: Actor Critic with lr_v = 0.2, lr_a = 0.01. With such a little knowledge of the state, the reference policy we built might seem as the optimal one. But here the surprise:

<img src="images/V1_0.2_0.01_-_(20,-0.2,-10).png" width=600>

As we can see, our algorithm finds even a better policy. In a nushell, it learns how to coil up, in order to reduce the risk of getting blocked within its own tail. 

This result was possible thanks to a careful tuning of the paramters. For example, if the learning rates are too high, the policy convinces itself of wrong strategies:

<img src="images/lr_troppo_alto_AC_0.9_0.09_-_(20,-0.2,-10).png" width=400>

<img src="images/V1_0.9_0.09_-_(20,-0.2,-10).png" width=400>


Here we can see that:\
    -the food is est\
    -there are blocks to the north and west\
    -the policy is collapsed and gives with probability 1 the action: go north

This example was performed with Actor Critic, lr_v = 0.9, lr_a = 0.09. The learining rates are indeed too large, and the learning is killed.

Also GAE performes well in this contex, for any value of $\lambda$:

<img src="images/V1_0.1_0.005_0.2_(20,-0.2,-10).png" width=400>

<img src="images/V1_0.05_0.003_0.9_(20,-0.2,-10).png" width=400>

We can see that for this binning bootstrapping is a good idea. In fact, since the states aren't much, the knowledge of them is valuable.

The last two plots give us the opportunity to point out that the laerning rates must be smaller for a larger $\lambda$. This comes from the fact that $\lambda$ increases variance, as discussed before. If, instead, we keep the same learning rates, the training is too unstable:

<img src="images/V1_0.1_0.005_0.9_(20,-0.2,-10).png" width=400>

#### Using Moore neighborhood radius 1

The next step in complexity is the Moore neighborhood. Using (*) once again, we obtain that the total number of bins, or states in our improper notation, is 2048. It turns out that the tabular algorithms developed in this section have already an hard time learining. We obtain a "not so good" result with GAE lr_v = 0.09 lr_a = 0.004 and Lambda = 0.2. We take the opportunity to show also the effect of something called "reward shaping". If we set reward food = 10, penality step = 0.5, penality death = 10, the learning is killed. On the left we see the original learning, and on the right the training with the same paramteres, but with the improper rewards:

<img src="images/M1_0.09_0.004_0.2_(20,-0.2,-10).png" width=400>

<img src="images/M1_0.09_0.004_0.2_(10,-0.5,-10).png" width=400>

#### Using Von Neuemann radius 2

Here we move even further down complexity of state representation. The number of total bins is 32 000. Clearely ufeasible for our tabular algorithms. Nonetheless we get to analyze better the difference between Actor Critic and GAE($\lambda$). 

With Actor Critic we use bootstrapping a lot, whereas with GAE($\lambda$) we use also the knowledge of future rewards, which are valueble fresh data from the enviroment. When the number of states was small, Actor Critic worked successfully because the value function is estimated well. When the states are a lot, we exept to have a less valuable estimation of the value function, making Actor Critic algorithms less efficient. In fact with Actor Critic, lr_v = 0.5, lr_a = 0.02 we have:

<img src="images/V2_0.5_0.02_-_(20,-0.2,-10).png" width=400>

Instead, we obtain a good result with GAE lr_v = 0.1 lr_a = 0.005 Lambda = 0.5, where we move towards a Monte Carlo approach: 

<img src="images/V2_0.1_0.005_0.5_(20,-0.2,-10).png" width=400>

We obtain surely a better result, but we are still far from the policy obtained with Von Neuemann radius 1, which scored up to 500. 

### **Hands on**

We leave here a ready-to-go code for trying out our best algorithms. First, excecute one of the two codes offered here, and then run the game to see the policy in action.
#### 1st : Actor Critic with Von Neumann radius 1
First we have the Actor Critic algorithm that carried out the best learning performance:

In [ ]:
# Bracketer
bracketer = NeighPlusFoodDirectionBracket(neigh="V", radius=1)

# General Settings 
gamma = 1         
lr_v = 0.2 
lr_a = 0.01     
n_episodes = 2501   

reward_food = 20      
reward_step = -0.2
reward_death = -10

#creating the enviroment
env = SnakeEnv(render_mode="nonhuman", max_step=10000, reward_food=reward_food, reward_step = reward_step, reward_death = reward_death)

#learning
algorithm = ActorCritic(env.action_space.n, gamma=gamma, lr_a=lr_a, bracketer=bracketer, lr_v=lr_v) 
algorithm.learning(env, None, n_episodes, bracketer)

#### 2nd: GAE(0.5) with Von Neumann radius 2

This neighborhood is quite complex, thus the learning might take a while.

In [ ]:
# Bracketer
bracketer = NeighPlusFoodDirectionBracket(neigh="V", radius=2)

# General Settings 
gamma = 1         
lr_v = 0.1 
lr_a = 0.005     
n_episodes = 30001   
Lambda = 0.5

reward_food = 20      
reward_step = -0.2
reward_death = -10

#creating the enviroment
env = SnakeEnv(render_mode="nonhuman", max_step=10000, reward_food=reward_food, reward_step = reward_step, reward_death = reward_death)

#learning
algorithm = GAE(env.action_space.n, gamma=gamma, lr_a=lr_a, bracketer=bracketer, lr_v=lr_v, Lambda=Lambda) 
algorithm.learning(env, None, n_episodes, bracketer)

#### Policy in action

Run this part after one of the two proposed learnings, to see the policy play.

In [ ]:
env = SnakeEnv(render_mode="human", max_step=10000, reward_food=reward_food, reward_step = reward_step, reward_death = reward_death)
algorithm.play(env, bracketer)